In [1]:
#clone YOLOv5 and
%cd yolov5
import torch
import os
from IPython.display import Image, clear_output  # to display images

/Users/ethanwu/Desktop/sambar/Deer_Detection/yolov5


In [2]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [3]:
import cv2
import numpy as np
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
%matplotlib inline
from IPython.display import clear_output, Image
import torch
import math


model = torch.hub.load('.', 'custom', path='/Users/ethanwu/Desktop/sambar/Deer_Detection/best.pt', source='local', force_reload=True)

cap = cv2.VideoCapture("/Users/ethanwu/Desktop/sambar/Deer_Detection/Deer_cctv5.mp4")

subtractor = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=170, detectShadows=True)

white_pixels_array = []

center_points_prev_frame = []

tracking_objects = {}

track_id = 0

count = 0

deleted_id = []

while True:
    clear_output(wait=True)
    ret, frame = cap.read()
    
    if count < 3:
        count += 1
    
    if not ret:
        break

    box_frame = model(frame) #np.squeeze(frame.render())
    mask = subtractor.apply(frame)

    white_pixels = 0 #np.sum(mask)
    
    center_points_cur_frame = []
    
    deer_dict = {}
    deer_index = 0
    for rect in box_frame.xyxy[0]:
        y1 = int(rect[1])
        y2 = int(rect[3])
        x1 = int(rect[0])
        x2 = int(rect[2])
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        # white_pixels += np.sum(mask[x1:x2, y1:y2]) # sum up all the pixels in the rectangle
        deer_dict["Deer " + str(deer_index)] = np.sum(mask[y1:y2, x1:x2])
        
        cv2.rectangle(mask, (int(rect[0]), int(rect[3])), (int(rect[2]), int(rect[1])), color=(198, 98,291), thickness=2) # print the rectangle
        cv2.putText(mask, "Motion: " + str(np.sum(mask[y1:y2, x1:x2])), (int(rect[0]), int(rect[1])), 1, 1, (255, 0, 0), 2)
        
        center_points_cur_frame.append([cx, cy, np.sum(mask[y1:y2, x1:x2])])
        deer_index += 1

    
    white_pixels_array.append(deer_dict)
    #print("Motion of Sambar Deer is: " + str(white_pixels) + " pixels")

    if count <= 2:
        for pt in center_points_cur_frame:
            for pt2 in center_points_prev_frame:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                if distance < 20:
                    tracking_objects[track_id] = pt
                    track_id += 1
    else:

        tracking_objects_copy = tracking_objects.copy()
        center_points_cur_frame_copy = center_points_cur_frame.copy()

        for object_id, pt2 in tracking_objects_copy.items():
            object_exists = False
            for pt in center_points_cur_frame_copy:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                # Update IDs position
                if distance < 20:
                    tracking_objects[object_id] = pt
                    object_exists = True
                    if pt in center_points_cur_frame:
                        center_points_cur_frame.remove(pt)
                    continue

            # Remove IDs lost
            if not object_exists:
                deleted_id.append(object_id)
                tracking_objects.pop(object_id)

        # Add new IDs found
        for pt in center_points_cur_frame:
            if len(deleted_id) != 0:
                tracking_objects[deleted_id.pop(-1)] = pt
            else:
                tracking_objects[track_id] = pt
                track_id += 1

    for object_id, pt in tracking_objects.items():
        cv2.circle(mask, (int(pt[0]), int(pt[1])), 5, (255, 0, 0), -1)
        cv2.putText(mask, str(object_id), (int(pt[0]), int(pt[1])), 0, 1, (255, 0, 0), 2)
    
    print(tracking_objects)
    
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)

    # Make a copy of the points
    center_points_prev_frame = center_points_cur_frame.copy()
    
    key = cv2.waitKey(30)

    if key == 27 & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 